In [1]:
## Without Duplicate Training Data
## Input features: Pressure, H2 to N2 ratio
import numpy as np
import pandas as pd
import itertools
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTENC, RandomOverSampler
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam

data2 = np.array([[0, 0.5, 0.36],
[20.49, 0.5, 0.69],
[39.18, 0.5, 0.96],
[59.59, 0.5, 1.21],
[79.93, 0.5, 1.43],
[0, 1, 0.58],
[10.24, 1, 0.79],
[20.11, 1, 0.96],
[39.33, 1, 1.23],
[59.44, 1, 1.44],
[75.44, 1, 1.55],
[79.85, 1, 1.57],
[0, 2, 0.60],
[10.09, 2, 0.78],
[20.41, 2, 0.94],
[39.18, 2, 1.15],
[59.51, 2, 1.29],
[75.59, 2, 1.34],
[80, 2, 1.34],
[0, 3, 0.61],
[10.32, 3, 0.71],
[20.56, 3, 0.79],
[39.18, 3, 0.91],
[59.44, 3, 1],
[79.85, 3, 1.03],
[0, 5, 0.51],
[10.32, 5, 0.56],
[20.49, 5, 0.59],
[39.33, 5, 0.65],
[59.66, 5, 0.67],
[79.85, 5, 0.68],
                    [80, 1, 1.57],
                    [80, 1, 1.16],
                    [80, 1, 1.02],
                    [80, 1, 1.89]])

# Convert data2 into a DataFrame
my_column_names = ['Pressure(psig)','H2 to N2 ratio','Ammonia Concentration(%)']
data2 = pd.DataFrame(data=data2, columns=my_column_names)

data2VariableNames = data2.columns.tolist()
feaColums = data2VariableNames[1:3]
labels = data2VariableNames[3:]

data2Del = data2.drop_duplicates(subset=feaColums,keep='first',inplace=False)
#print(data2Del)

ind_list = [i for i in range(data2Del.shape[0])]
#print(ind_list)

ind_set = list(itertools.combinations(ind_list,2))
#print(ind_set)

model_smote = SMOTE(k_neighbors=1,random_state=0)

data_smote_all = []
ind_smote_all = []
ind_smote = np.zeros(data2Del.shape[0]-5)
#print(ind_smote)

ind_smote[:2] = 1
#print(ind_smote[:2])
smote_data = []
i = 0

for item in ind_set:
    ind_ = list(item)
    y_smote = np.zeros(data2Del.shape[0])
    y_smote[ind_] = 1
    i +=1
    data_smote_resampled, y_smote_resampled = model_smote.fit_resample(data2Del, y_smote)
    smote_data.append(data_smote_resampled)

# Convert the list to a NumPy array
smote_data = np.array(smote_data)
smote_data = smote_data.reshape(-1, 3)
#print(smote_data.shape)


# Convert the array to a list of tuples to preserve the 2D structure
smote_data_tuples = [tuple(row) for row in smote_data]

# Storing unique data points
unique_tuples = np.unique(smote_data_tuples, axis=0)

# Convert the unique tuples back to a NumPy array
smote_data_array = np.array(unique_tuples)
#print(smote_data_array .shape)
smote_data_feature = smote_data_array[:, :2]
smote_data_label = smote_data_array[:, 2]

# Add random perturbations to the target variable
synthetic_y = smote_data_array[:, 2]

## Establishing Neural Network
# define the deep neural network model
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))


# compile the model
opt = Adam(learning_rate=0.0001)
model.compile(loss='mse', optimizer=opt)

# train the model on the training set
model.fit(smote_data_feature, synthetic_y, epochs=100, batch_size=3)
history_train = model.fit(smote_data_feature, synthetic_y, epochs=100, batch_size=3)

# Constant values
# Temperature = 320
# Gas_flow_rate = 100

# pressure range with an increment of 1
Pressure_range = range(0, 81, 1)

###############################################################################
Ratio_Exp = 0.5

# Creating array for input features
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=0.5')
plt.scatter([0,20,40,80],[0.4,0.63,0.99,1.41])
plt.xlabel('Pressure(psig)')
plt.ylabel('NH3 concentration(%)')
plt.legend()

###############################################################################
Ratio_Exp = 1
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=1')
plt.scatter([0,20,40,60,75,80],[0.57,0.95,1.24,1.43,1.55,1.57])
plt.legend()
###############################################################################
Ratio_Exp = 2
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=2')
plt.scatter([0,20,40,60,75,80],[0.6,0.96,1.16,1.27,1.33,1.35])
plt.legend()
###############################################################################
Ratio_Exp = 3
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=3')
plt.scatter([0,20,40,80],[0.56,0.85,0.89,1.06])
plt.legend()
###############################################################################
Ratio_Exp = 5
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=5')
plt.scatter([0,20,40,80],[0.48,0.63,0.63,0.69])
plt.legend()


Epoch 1/100


c:\Users\japan\anaconda3\envs\ARIM\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5115/5115 ━━━━━━━━━━━━━━━━━━━━ 6s 882us/step - loss: 0.5970
Epoch 2/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 920us/step - loss: 0.0311
Epoch 3/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 941us/step - loss: 0.0265
Epoch 4/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 939us/step - loss: 0.0241
Epoch 5/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 974us/step - loss: 0.0217
Epoch 6/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.0214
Epoch 7/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 969us/step - loss: 0.0201
Epoch 8/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 972us/step - loss: 0.0195
Epoch 9/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 5s 951us/step - loss: 0.0190
Epoch 10/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - loss: 0.0178
Epoch 11/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0177
Epoch 12/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0173
Epoch 13/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0174
Epoch 14/100
5115/5115 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 0.0174
Epoch 15/

ValueError: Unrecognized data type: x=[[0, 0.5], [1, 0.5], [2, 0.5], [3, 0.5], [4, 0.5], [5, 0.5], [6, 0.5], [7, 0.5], [8, 0.5], [9, 0.5], [10, 0.5], [11, 0.5], [12, 0.5], [13, 0.5], [14, 0.5], [15, 0.5], [16, 0.5], [17, 0.5], [18, 0.5], [19, 0.5], [20, 0.5], [21, 0.5], [22, 0.5], [23, 0.5], [24, 0.5], [25, 0.5], [26, 0.5], [27, 0.5], [28, 0.5], [29, 0.5], [30, 0.5], [31, 0.5], [32, 0.5], [33, 0.5], [34, 0.5], [35, 0.5], [36, 0.5], [37, 0.5], [38, 0.5], [39, 0.5], [40, 0.5], [41, 0.5], [42, 0.5], [43, 0.5], [44, 0.5], [45, 0.5], [46, 0.5], [47, 0.5], [48, 0.5], [49, 0.5], [50, 0.5], [51, 0.5], [52, 0.5], [53, 0.5], [54, 0.5], [55, 0.5], [56, 0.5], [57, 0.5], [58, 0.5], [59, 0.5], [60, 0.5], [61, 0.5], [62, 0.5], [63, 0.5], [64, 0.5], [65, 0.5], [66, 0.5], [67, 0.5], [68, 0.5], [69, 0.5], [70, 0.5], [71, 0.5], [72, 0.5], [73, 0.5], [74, 0.5], [75, 0.5], [76, 0.5], [77, 0.5], [78, 0.5], [79, 0.5], [80, 0.5]] (of type <class 'list'>)

In [ ]:
## Without Duplicate Training Data
## Input features: Pressure, H2 to N2 ratio
import numpy as np
import pandas as pd
import itertools
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTENC, RandomOverSampler
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam

data2 = np.array([[0, 0.5, 0.36],
[20.49, 0.5, 0.69],
[39.18, 0.5, 0.96],
[59.59, 0.5, 1.21],
[79.93, 0.5, 1.43],
[0, 1, 0.58],
[10.24, 1, 0.79],
[20.11, 1, 0.96],
[39.33, 1, 1.23],
[59.44, 1, 1.44],
[75.44, 1, 1.55],
[79.85, 1, 1.57],
[0, 2, 0.60],
[10.09, 2, 0.78],
[20.41, 2, 0.94],
[39.18, 2, 1.15],
[59.51, 2, 1.29],
[75.59, 2, 1.34],
[80, 2, 1.34],
[0, 3, 0.61],
[10.32, 3, 0.71],
[20.56, 3, 0.79],
[39.18, 3, 0.91],
[59.44, 3, 1],
[79.85, 3, 1.03],
[0, 5, 0.51],
[10.32, 5, 0.56],
[20.49, 5, 0.59],
[39.33, 5, 0.65],
[59.66, 5, 0.67],
[79.85, 5, 0.68],
                    [80, 1, 1.57],
                    [80, 1, 1.16],
                    [80, 1, 1.02],
                    [80, 1, 1.89]])

# Convert data2 into a DataFrame
my_column_names = ['Pressure(psig)','H2 to N2 ratio','Ammonia Concentration(%)']
data2 = pd.DataFrame(data=data2, columns=my_column_names)

data2VariableNames = data2.columns.tolist()
feaColums = data2VariableNames[1:3]
labels = data2VariableNames[3:]

data2Del = data2.drop_duplicates(subset=feaColums,keep='first',inplace=False)
#print(data2Del)

ind_list = [i for i in range(data2Del.shape[0])]
#print(ind_list)

ind_set = list(itertools.combinations(ind_list,2))
#print(ind_set)

model_smote = SMOTE(k_neighbors=1,random_state=0)

data_smote_all = []
ind_smote_all = []
ind_smote = np.zeros(data2Del.shape[0]-5)
#print(ind_smote)

ind_smote[:2] = 1
#print(ind_smote[:2])
smote_data = []
i = 0

for item in ind_set:
    ind_ = list(item)
    y_smote = np.zeros(data2Del.shape[0])
    y_smote[ind_] = 1
    i +=1
    data_smote_resampled, y_smote_resampled = model_smote.fit_resample(data2Del, y_smote)
    smote_data.append(data_smote_resampled)

# Convert the list to a NumPy array
smote_data = np.array(smote_data)
smote_data = smote_data.reshape(-1, 3)
#print(smote_data.shape)


# Convert the array to a list of tuples to preserve the 2D structure
smote_data_tuples = [tuple(row) for row in smote_data]

# Storing unique data points
unique_tuples = np.unique(smote_data_tuples, axis=0)

# Convert the unique tuples back to a NumPy array
smote_data_array = np.array(unique_tuples)
#print(smote_data_array .shape)
smote_data_feature = smote_data_array[:, :2]
smote_data_label = smote_data_array[:, 2]

# Add random perturbations to the target variable
synthetic_y = smote_data_array[:, 2]

## Establishing Neural Network
# define the deep neural network model
model = Sequential()
model.add(Dense(64, input_dim=2, activation='softplus'))
model.add(Dense(128, activation='softplus'))
model.add(Dense(64, activation='softplus'))
model.add(Dense(1, activation='linear'))


# compile the model
opt = Adam(learning_rate=0.0001)
model.compile(loss='mse', optimizer=opt)

# train the model on the training set
model.fit(smote_data_feature, synthetic_y, epochs=100, batch_size=3)
history_train = model.fit(smote_data_feature, synthetic_y, epochs=100, batch_size=3)

# Constant values
# Temperature = 320
# Gas_flow_rate = 100

# pressure range with an increment of 1
Pressure_range = range(0, 81, 1)

###############################################################################
Ratio_Exp = 0.5

# Creating array for input features
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=0.5')
plt.scatter([0,20,40,80],[0.4,0.63,0.99,1.41])
plt.xlabel('Pressure(psig)')
plt.ylabel('NH3 concentration(%)')
plt.legend()

###############################################################################
Ratio_Exp = 1
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=1')
plt.scatter([0,20,40,60,75,80],[0.57,0.95,1.24,1.43,1.55,1.57])
plt.legend()
###############################################################################
Ratio_Exp = 2
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=2')
plt.scatter([0,20,40,60,75,80],[0.6,0.96,1.16,1.27,1.33,1.35])
plt.legend()
###############################################################################
Ratio_Exp = 3
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=3')
plt.scatter([0,20,40,80],[0.56,0.85,0.89,1.06])
plt.legend()
###############################################################################
Ratio_Exp = 5
Combined_array = []

for Pressure in Pressure_range:
    Combined_array.append([Pressure, Ratio_Exp])

z_pred = model.predict(Combined_array)

# Extract pressure, ratio, and temperature from Combined_array
pressure = [row[0] for row in Combined_array]
ratio = [row[1] for row in Combined_array]

# Create 2D figure
fig = plt.plot(pressure, z_pred, label='Ratio=5')
plt.scatter([0,20,40,80],[0.48,0.63,0.63,0.69])
plt.legend()
